<a href="https://colab.research.google.com/github/Witcape/Emotion_Recognition_in_Live_Video_Data/blob/main/Emotion_Recognition_in_Video_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Importing all the required libraries
from sympy import *
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import sklearn
import tensorflow as tf
import pandas as pd
import sklearn
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import linear,relu,sigmoid

In [ ]:
x_demo = [15,22,32,44,75,6,7,8,9,11,23,4,21,234,34234,123,122,3,4,21,]
x_demo = np.array(x_demo)
y_demo = [253,234,2362,36,2546,45,24235,23,5,654,45,23,45,234,5,2345,256,2456,4,5]
y_demo = np.array(y_demo)

In [ ]:
model = Sequential(
    [
        tf.keras.Input(shape = (20,)),
        Dense(10, activation = relu),
        Dense(1, activation = linear)
    ]
)


In [ ]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_19 (Dense)            (None, 10)                210       
                                                                 
 dense_20 (Dense)            (None, 1)                 11        
                                                                 
Total params: 221
Trainable params: 221
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(
   optimizer='adam',
   loss='mean_squared_error'
   )
history = model.fit(
    x_demo,y_demo,
    epochs=40
)

Epoch 1/40


ValueError: ignored